# Quantitative Momentum Strategy Carlos

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [7]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [8]:
stocks = pd.read_csv('sp_my_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2434857562901,
 'week52high': 153.73,
 'week52low': 91.42,
 'week52highSplitAdjustOnly': 149.96,
 'week52lowSplitAdjustOnly': 92.37,
 'week52change': 0,
 'sharesOutstanding': 17103130437,
 'float': 0,
 'avg10Volume': 101428031,
 'avg30Volume': 85063033,
 'day200MovingAvg': 132.37,
 'day50MovingAvg': 134.29,
 'employees': 148186,
 'ttmEPS': 4.61,
 'ttmDividendRate': 0.8495298473271599,
 'dividendYield': 0.00580425872839445,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-21',
 'peRatio': 32.3604960982472,
 'beta': 1.6162703625072876,
 'maxChangePercent': 57.40828875942302,
 'year5ChangePercent': 5.44417253342455,
 'year2ChangePercent': 1.9742959561879292,
 'year1ChangePercent': 0.5533688689908917,
 'ytdChangePercent': 0.10377433695882826,
 'month6ChangePercent': 0.1173724841933364,
 'month3ChangePercent': 0.0863595143984785,
 'month1ChangePercent': 0.1456297262509177,
 'day30ChangePercent': 0.150113194958

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [10]:
data['year1ChangePercent']

0.5533688689908917

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,156.563,0.542847,N/A
1,AMZN,3728.600,0.191690,N/A
2,EBAY,71.505,0.203303,N/A
3,GOOGL,2682.040,0.686958,N/A
4,IBM,141.570,0.249975,N/A
5,LHX,224.740,0.416167,N/A
6,NFLX,564.190,0.030172,N/A
7,NI,26.000,0.079423,N/A
8,NKE,167.930,0.696898,N/A
9,NVDA,804.290,1.051516,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
#not doing this now, since I only have a few
#final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
#final_dataframe = final_dataframe[:51]
#final_dataframe.reset_index(drop = True, inplace = True)
#final_dataframe

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [11]:
portfolio_size=10000
def portfolio_input(portfolio_size):
    #global portfolio_size
    #portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input(portfolio_size)
#print(portfolio_size)

In [12]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,147.170,0.527049,6
1,AMZN,3800.500,0.163351,0
2,EBAY,71.340,0.181102,14
3,GOOGL,2557.160,0.654991,0
4,IBM,143.770,0.261962,6
5,LHX,232.760,0.411471,4
6,NFLX,561.108,-0.021852,1
7,NI,25.606,0.110147,39
8,NKE,165.820,0.672366,6
9,NVDA,811.390,0.985205,1


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [14]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0    0.7
1    0.3
2    0.4
3    0.9
4    0.5
5    0.6
6    0.1
7    0.2
8    0.8
9      1
Name: One-Year Return Percentile, dtype: object
0    0.4
1    0.6
2    0.8
3    0.9
4    0.2
5    0.7
6    0.1
7    0.5
8    0.3
9      1
Name: Six-Month Return Percentile, dtype: object
0    0.6
1    0.5
2    0.7
3    0.8
4    0.3
5    0.4
6    0.1
7    0.2
8    0.9
9      1
Name: Three-Month Return Percentile, dtype: object
0    0.8
1    0.7
2    0.4
3    0.5
4    0.1
5    0.3
6    0.6
7    0.2
8      1
9    0.9
Name: One-Month Return Percentile, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,147.294,N/A,0.529334,0.7,0.131448,0.4,0.104773,0.6,0.139171,0.8,N/A
1,AMZN,3879.700,N/A,0.162430,0.3,0.199820,0.6,0.102504,0.5,0.116270,0.7,N/A
2,EBAY,70.900,N/A,0.181580,0.4,0.243517,0.8,0.107880,0.7,0.028080,0.4,N/A
3,GOOGL,2645.710,N/A,0.673398,0.9,0.465080,0.9,0.137629,0.8,0.045271,0.5,N/A
4,IBM,146.984,N/A,0.265532,0.5,0.119765,0.2,0.059666,0.3,-0.068613,0.1,N/A
5,LHX,232.100,N/A,0.414280,0.6,0.231312,0.7,0.076895,0.4,0.004429,0.3,N/A
6,NFLX,555.980,N/A,-0.021290,0.1,0.088740,0.1,-0.028128,0.1,0.099524,0.6,N/A
7,NI,26.159,N/A,0.114351,0.2,0.175875,0.5,0.029678,0.2,-0.030740,0.2,N/A
8,NKE,165.330,N/A,0.672025,0.8,0.120997,0.3,0.191724,0.9,0.235144,1,N/A
9,NVDA,808.035,N/A,0.997996,1,0.527947,1,0.362007,1,0.158003,0.9,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [15]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [16]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe#[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [17]:
portfolio_input(portfolio_size)

In [18]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,147.294,6,0.529334,0.7,0.131448,0.4,0.104773,0.6,0.139171,0.8,0.625
1,AMZN,3879.700,0,0.162430,0.3,0.199820,0.6,0.102504,0.5,0.116270,0.7,0.525
2,EBAY,70.900,14,0.181580,0.4,0.243517,0.8,0.107880,0.7,0.028080,0.4,0.575
3,GOOGL,2645.710,0,0.673398,0.9,0.465080,0.9,0.137629,0.8,0.045271,0.5,0.775
4,IBM,146.984,6,0.265532,0.5,0.119765,0.2,0.059666,0.3,-0.068613,0.1,0.275
5,LHX,232.100,4,0.414280,0.6,0.231312,0.7,0.076895,0.4,0.004429,0.3,0.5
6,NFLX,555.980,1,-0.021290,0.1,0.088740,0.1,-0.028128,0.1,0.099524,0.6,0.225
7,NI,26.159,38,0.114351,0.2,0.175875,0.5,0.029678,0.2,-0.030740,0.2,0.275
8,NKE,165.330,6,0.672025,0.8,0.120997,0.3,0.191724,0.9,0.235144,1,0.75
9,NVDA,808.035,N/A,0.997996,1,0.527947,1,0.362007,1,0.158003,0.9,0.975


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [19]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [20]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [21]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [22]:
writer.save()